In [ ]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
print(0)

0
time: 526 µs (started: 2022-02-23 18:06:34 +00:00)


In [14]:
CUR_PATH='/home/karanverma/test_task_time_slice/'
import sys
sys.path.insert(1, CUR_PATH)
sys.path.insert(1, CUR_PATH+'/TimeSliceEvaluator')
CUR_PATH='/home/karanverma/test_task_time_slice/'

time: 584 µs (started: 2022-02-23 18:41:19 +00:00)


In [15]:
import datetime
import numpy as np
from tqdm import tqdm
import pickle
import pandas as pd

df = pd.read_csv(CUR_PATH + "data_filtered.csv")
df = df[['deviceId',"overallTimeSpent",'eventTimestamp','hashId']]
df = df.sort_values(by=['eventTimestamp'])
df['eventTimestamp'] /= 1000
newsContent = pickle.load(open(CUR_PATH+'/news_arr.pickle', 'rb'))

def mapValues(values):
    return {"deviceId" : values[0], "hashId" : values[3], "overallTimeSpent" : values[1], "eventTimestamp" : values[4]}

stream = df.to_numpy()

columns = ['hashId', 'timestamp']
array = []
for n in newsContent:
    t = newsContent[n]['publish_time']
    array.append([n, int(t)])

newsDf = pd.DataFrame(data=array, columns=columns)

time: 7.7 s (started: 2022-02-23 18:41:21 +00:00)


In [5]:
import matplotlib.pyplot as plt
import random

time: 705 ms (started: 2022-02-23 18:07:05 +00:00)


In [6]:
import RankEvaluator
from importlib import reload
reload(RankEvaluator)

<module 'RankEvaluator' from '/home/karanverma/test_task_time_slice/RankEvaluator.py'>

time: 4.83 ms (started: 2022-02-23 18:07:25 +00:00)


In [7]:
PolicyEvaluator = RankEvaluator.TimeSliceEvaluatorGenerator(df, newsContent, sliceSize=1, verbose=False)

time: 378 µs (started: 2022-02-23 18:07:28 +00:00)


## Time Slice Evaluator

Below is the description of Recommender Evaluator mechanism, which can be used to evaluate a recommendation algorithm.


#### Problem Statement (aka what are we trying to solve)

We want to solve the recommendation challenge for users. That is, given some history of users reading pattern, we want to recommend new news shorts for the user on which user will spend maximum time. Formally, we have a history as a list of tuples (user, news, timespent)<sub>t</sub> ordered by timestamp t, from which we have to learn the interests of the users and recommend news accordingly.

#### Evaluation


The evaluation works as follows. The Recommender algorithm needs to implements two functions 1. <code>recommend</code> and 2. <code>feed</code>

Recommend takes a list of pairs (each pair consisting of a user and news). It should return a list of scores for the pairs in the list pairs passed ( the score should have the property that if user A likes news N1 more than N2, then score(A,N1) should be higher than score(A,N2) ). 

Feed method then is given the a list of events <code>news, user, timespent</code> which contains the feedback for your recommendation. The data for this stream is generated using the file data_filtered.csv. The recommender algorithm implemented by you, will be run back and forth as recommend and feed. 

Follwing are the sample paramenters for recommend and feed methods.

P.recommend(['7sgdbhkd-cfd6-460d-b785-ffd028a02c27','8uudbhkd-cfd6-460d-b785-ffd028a02c27','94hjbhkd-cfd6-460d-b785-ffd028a0bgf27'],['9na5zqtz-1','67na5zqtz-1','0gtfqtz-1'],NewsContent)
where news content is a dict containing a map from newsId to it's meta data like text, category, publisher, etc.
The return value would we a list of length threee for above inputs, which will the score for the pairs [(user_list[0],news_list[0]),(user_list[1],news_list[1]),(user_list[2],news_list[2])]

P.feed([{'deviceId': '7sgdbhkd-cfd6-460d-b785-ffd028a02c27', 'hashId': '9na5zqtz-1', 'eventTimestamp': 1624134990.0, 'overallTimeSpent': 3.556},{'deviceId': '9cf6e7af-cfd6-460d-b785-ffd028a02c27', 'hashId': '564h45tz-1', 'eventTimestamp': 1624147990.0, 'overallTimeSpent': 6.556}])


#### Scoring

The scoring of the recomender system is done by comparing the order of the news of a particular user according to the scores by recomender vs the order of the news for the same user according to the known timespents. 

#### Benchmarks

We have given a sample algorithm "RandomRecommenderPolicy", which generates random score for each user,news pair. Ideally, the expected score for a recommender that gives random results would be 50. However, Due to the finite size of the data set, the value we are getting for random recommender is 55. The score that our popularity based recommender is able to afford is around 50, so you should be targetting this score. 

#### Approach

We would be reading and reviewing your code, so please document your approach very well. Half of the weightage for selection will given to the documentation and clarity of the approach. You are not allowed to make any changes in the acommpanied file or anything pre-written in this notebook. You will be given a clean section and you can show your work there.

time: 3.38 ms (started: 2022-01-21 13:10:55 +00:00)


## Default Policy

<code>RandomRecommenderPolicy</code> is a default recommendation policy class given in RankEvaluator.py file, which just generates a random score for each user_list and news_list pair. We have shown here how it works and the score it achieves on our policy evaluator.

In [8]:
from RankEvaluator import *

time: 372 µs (started: 2022-02-23 18:08:39 +00:00)


In [11]:
recPolicy = RandomRecommenderPolicy()
score = PolicyEvaluator(recPolicy)
print(score)

100%|██████████| 48/48 [00:00<00:00, 52.18it/s]

{'score': 55.49954052319936, 'score_list': [52.57585874661267, 54.18936816810165, 55.106692348404295, 55.19234967584315, 56.4740711545418, 56.04354770755631, 55.41407556270722, 56.08484625212916, 57.37681894032903, 55.78281664651213, 55.82642108892245, 55.696229956640565, 55.90396430222043, 55.585336370741615, 57.485378490847964, 54.58210381727192, 55.128436625314734, 53.785115752353285, 52.903265460316916, 51.560156795531164, 51.69143542267914, 50.23429717384189, 52.63205666126222, 53.06369604976896, 53.746800073832986, 53.03874463200516, 55.36055880392329, 53.752957445033104, 55.79276505071512, 55.577346250099666, 56.91654640005843, 53.73879383996587, 57.02159061573071, 57.029607465424824, 56.31518238037467, 58.91775846588787, 55.128475557363274, 54.66257171451877, 56.39130225562741, 55.532568965567904, 54.479442194641436, 55.178728837454784, 52.216873498452024, 51.631861206114316, 68.2278053557017, 52.114492846047526, 53.14262652793534]}
time: 1 s (started: 2022-02-23 18:21:17 +00:0

#### Your Policy Training Goes Here